In [8]:
import os

In [9]:
%pwd

'c:\\Users\\pouru\\OneDrive\\Desktop\\360_digit_mg_solution\\360_digit_mg_project\\Fair-and-Efficient-Bonus-Allocation-System\\Research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\pouru\\OneDrive\\Desktop\\360_digit_mg_solution\\360_digit_mg_project\\Fair-and-Efficient-Bonus-Allocation-System'

In [46]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str


In [47]:
from src.Bonus_Allocation_System.constants import *
from src.Bonus_Allocation_System.utils.common import read_yaml,create_directories

In [48]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.KNeighborsClassifier
        schema =  self.schema.TARGET_COLUMNS

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,

            
        )

        return model_trainer_config

In [49]:
import pandas as pd
import os
from src.Bonus_Allocation_System.logging import logger
from sklearn.neighbors import KNeighborsClassifier
import joblib

In [50]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        train_data.head()


        train_x = train_data[['Winning_percentage', 'Average_Bet_Amount',
       'Number_of_Bonuses_Received', 'Amount_of_Bonuses_Received',
       'Revenue_from_Bonuses']]
        test_x = test_data[['Winning_percentage', 'Average_Bet_Amount',
       'Number_of_Bonuses_Received', 'Amount_of_Bonuses_Received',
       'Revenue_from_Bonuses']]
        train_y = train_data[['Should_Receive_Bonus']]
        test_y = test_data[['Should_Receive_Bonus']]


        lr = KNeighborsClassifier()
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [7]:
import os 
%pwd

'c:\\Users\\pouru\\OneDrive\\Desktop\\360_digit_mg_solution\\360_digit_mg_project\\Fair-and-Efficient-Bonus-Allocation-System\\Research'

In [51]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-14 15:51:36,503: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-14 15:51:36,507: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-14 15:51:36,516: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-14 15:51:36,518: INFO: common: created directory at: artifacts]
[2024-08-14 15:51:36,522: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\pouru\anaconda3\envs\bonus_allocation_system\lib\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
